Tugas 1

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min

spark = SparkSession.builder.appName("HandsOnPertemuan3").getOrCreate()

data = [
    ('Kim Dokja', 'Karyawan Kantoran', 3500),
    ('Han Sooyoung', 'Penulis Webnovel', 2800),
    ('Yoo Joonghyuk', 'Gamer Profesional', 4000),
    ('Lee Hyunsung', 'Tentara', 3700),
    ('Jung Heewon', 'Pengacara', 3900),
    ('Lee Jihye', 'Pelajar', 0),
    ('Shin Yoosung', 'Pelajar', 0)
]
columns = ['CharacterName', 'OccupationBefore', 'Salary']

df = spark.createDataFrame(data, schema=columns)

print("~~~ Data Awal ~~~")
df.show()

print("~~~ Schema ~~~")
df.printSchema()

print("~~~ Nama dan Gaji ~~~")
df.select("CharacterName", "Salary").show()

print("~~~ Update Salary +500 ~~~")
df.withColumn("UpdatedSalary", col("Salary") + 500).show()

print("~~~ Karakter dengan gaji > 3000 ~~~")
df.filter(col("Salary") > 3000).show()

print("~~~ Rata-rata gaji per pekerjaan ~~~")
df.groupBy("OccupationBefore").agg(avg("Salary").alias("AverageSalary")).show()

print("~~~ Gaji maksimum & minimum per pekerjaan ~~~")
df.groupBy("OccupationBefore").agg(
    max("Salary").alias("MaxSalary"),
    min("Salary").alias("MinSalary")
).show()

~~~ Data Awal ~~~
+-------------+-----------------+------+
|CharacterName| OccupationBefore|Salary|
+-------------+-----------------+------+
|    Kim Dokja|Karyawan Kantoran|  3500|
| Han Sooyoung| Penulis Webnovel|  2800|
|Yoo Joonghyuk|Gamer Profesional|  4000|
| Lee Hyunsung|          Tentara|  3700|
|  Jung Heewon|        Pengacara|  3900|
|    Lee Jihye|          Pelajar|     0|
| Shin Yoosung|          Pelajar|     0|
+-------------+-----------------+------+

~~~ Schema ~~~
root
 |-- CharacterName: string (nullable = true)
 |-- OccupationBefore: string (nullable = true)
 |-- Salary: long (nullable = true)

~~~ Nama dan Gaji ~~~
+-------------+------+
|CharacterName|Salary|
+-------------+------+
|    Kim Dokja|  3500|
| Han Sooyoung|  2800|
|Yoo Joonghyuk|  4000|
| Lee Hyunsung|  3700|
|  Jung Heewon|  3900|
|    Lee Jihye|     0|
| Shin Yoosung|     0|
+-------------+------+

~~~ Update Salary +500 ~~~
+-------------+-----------------+------+-------------+
|CharacterName| Occupa

Tugas 2

In [3]:
df.select('CharacterName', 'Salary').show()

df.filter(df['Salary'] > 3000).show()

df.groupBy('OccupationBefore').avg('Salary').show()

df.groupBy('OccupationBefore').max('Salary').show()

df.groupBy('OccupationBefore').sum('Salary').show()

+-------------+------+
|CharacterName|Salary|
+-------------+------+
|    Kim Dokja|  3500|
| Han Sooyoung|  2800|
|Yoo Joonghyuk|  4000|
| Lee Hyunsung|  3700|
|  Jung Heewon|  3900|
|    Lee Jihye|     0|
| Shin Yoosung|     0|
+-------------+------+

+-------------+-----------------+------+
|CharacterName| OccupationBefore|Salary|
+-------------+-----------------+------+
|    Kim Dokja|Karyawan Kantoran|  3500|
|Yoo Joonghyuk|Gamer Profesional|  4000|
| Lee Hyunsung|          Tentara|  3700|
|  Jung Heewon|        Pengacara|  3900|
+-------------+-----------------+------+

+-----------------+-----------+
| OccupationBefore|avg(Salary)|
+-----------------+-----------+
|Gamer Profesional|     4000.0|
|Karyawan Kantoran|     3500.0|
| Penulis Webnovel|     2800.0|
|          Tentara|     3700.0|
|        Pengacara|     3900.0|
|          Pelajar|        0.0|
+-----------------+-----------+

+-----------------+-----------+
| OccupationBefore|max(Salary)|
+-----------------+-----------+


Tugas 3

In [6]:
df = df.withColumn('SalaryBonus', df['Salary'] * 0.1)

df = df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus'])

df = df.withColumn('Profile', concat_ws(" - ", col("CharacterName"), col("OccupationBefore")))

df = df.withColumn('StructData', struct("CharacterName", "Salary"))

df = df.withColumn('ArrayData', array("CharacterName", "OccupationBefore"))

df.show(truncate=False)

+-------------+-----------------+------+-----------+-----------------+---------------------------------+---------------------+----------------------------------+
|CharacterName|OccupationBefore |Salary|SalaryBonus|TotalCompensation|Profile                          |StructData           |ArrayData                         |
+-------------+-----------------+------+-----------+-----------------+---------------------------------+---------------------+----------------------------------+
|Kim Dokja    |Karyawan Kantoran|3500  |350.0      |3850.0           |Kim Dokja - Karyawan Kantoran    |{Kim Dokja, 3500}    |[Kim Dokja, Karyawan Kantoran]    |
|Han Sooyoung |Penulis Webnovel |2800  |280.0      |3080.0           |Han Sooyoung - Penulis Webnovel  |{Han Sooyoung, 2800} |[Han Sooyoung, Penulis Webnovel]  |
|Yoo Joonghyuk|Gamer Profesional|4000  |400.0      |4400.0           |Yoo Joonghyuk - Gamer Profesional|{Yoo Joonghyuk, 4000}|[Yoo Joonghyuk, Gamer Profesional]|
|Lee Hyunsung |Tentara      

Tugas 4

In [7]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('OccupationBefore').orderBy('Salary')

df_rank = df.withColumn('Rank', F.rank().over(windowSpec))

df_running = df.withColumn('RunningTotal', F.sum('Salary').over(windowSpec))

df_dense = df.withColumn('DenseRank', F.dense_rank().over(windowSpec))

print("~~~ Ranking gaji per pekerjaan ~~~")
df_rank.show()

print("~~~ Running total gaji per pekerjaan ~~~")
df_running.show()

print("~~~ Dense Rank gaji per pekerjaan ~~~")
df_dense.show()

~~~ Ranking gaji per pekerjaan ~~~
+-------------+-----------------+------+-----------+-----------------+--------------------+--------------------+--------------------+----+
|CharacterName| OccupationBefore|Salary|SalaryBonus|TotalCompensation|             Profile|          StructData|           ArrayData|Rank|
+-------------+-----------------+------+-----------+-----------------+--------------------+--------------------+--------------------+----+
|Yoo Joonghyuk|Gamer Profesional|  4000|      400.0|           4400.0|Yoo Joonghyuk - G...|{Yoo Joonghyuk, 4...|[Yoo Joonghyuk, G...|   1|
|    Kim Dokja|Karyawan Kantoran|  3500|      350.0|           3850.0|Kim Dokja - Karya...|   {Kim Dokja, 3500}|[Kim Dokja, Karya...|   1|
|    Lee Jihye|          Pelajar|     0|        0.0|              0.0| Lee Jihye - Pelajar|      {Lee Jihye, 0}|[Lee Jihye, Pelajar]|   1|
| Shin Yoosung|          Pelajar|     0|        0.0|              0.0|Shin Yoosung - Pe...|   {Shin Yoosung, 0}|[Shin Yoosung, Pe..

Tugas 5

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NetflixHandsOn").getOrCreate()

df_netflix = spark.read.csv("netflix_titles.csv", header=True, inferSchema=True)

In [20]:
print("Schema DataFrame:")
df_netflix.printSchema()

print("Jumlah total data:", df_netflix.count())

print("Preview data:")
df_netflix.show(5)

Schema DataFrame:
root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

Jumlah total data: 8809
Preview data:
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------

In [21]:
# Memilih kolom tertentu
df_netflix.select("title", "type", "release_year").show(5, truncate=False)

# Filter film setelah tahun 2015
df_netflix.filter(df_netflix.release_year > 2015) \
          .select("title", "release_year") \
          .show(5, truncate=False)

# GroupBy + Aggregasi
df_netflix.groupBy("country").count().orderBy("count", ascending=False).show(10, truncate=False)
df_netflix.groupBy("release_year", "type").count().orderBy("release_year", ascending=False).show(10, truncate=False)

+---------------------+-------+------------+
|title                |type   |release_year|
+---------------------+-------+------------+
|Dick Johnson Is Dead |Movie  |2020        |
|Blood & Water        |TV Show|2021        |
|Ganglands            |TV Show|2021        |
|Jailbirds New Orleans|TV Show|2021        |
|Kota Factory         |TV Show|2021        |
+---------------------+-------+------------+
only showing top 5 rows

+---------------------+------------+
|title                |release_year|
+---------------------+------------+
|Dick Johnson Is Dead |2020        |
|Blood & Water        |2021        |
|Ganglands            |2021        |
|Jailbirds New Orleans|2021        |
|Kota Factory         |2021        |
+---------------------+------------+
only showing top 5 rows

+--------------+-----+
|country       |count|
+--------------+-----+
|United States |2805 |
|India         |972  |
|NULL          |832  |
|United Kingdom|419  |
|Japan         |245  |
|South Korea   |199  |
|Cana

In [22]:
from pyspark.sql.functions import length, split, explode

# Tambahkan kolom panjang judul
df_netflix = df_netflix.withColumn("title_length", length("title"))
df_netflix.select("title", "title_length").show(5, truncate=False)

# Pecah genre menjadi array lalu explode → satu genre per baris
df_genre = df_netflix.withColumn("genre", explode(split("listed_in", ", ")))
df_genre.select("title", "genre").show(10, truncate=False)

+---------------------+------------+
|title                |title_length|
+---------------------+------------+
|Dick Johnson Is Dead |20          |
|Blood & Water        |13          |
|Ganglands            |9           |
|Jailbirds New Orleans|21          |
|Kota Factory         |12          |
+---------------------+------------+
only showing top 5 rows

+---------------------+----------------------+
|title                |genre                 |
+---------------------+----------------------+
|Dick Johnson Is Dead |Documentaries         |
|Blood & Water        |International TV Shows|
|Blood & Water        |TV Dramas             |
|Blood & Water        |TV Mysteries          |
|Ganglands            |Crime TV Shows        |
|Ganglands            |International TV Shows|
|Ganglands            |TV Action & Adventure |
|Jailbirds New Orleans|Docuseries            |
|Jailbirds New Orleans|Reality TV            |
|Kota Factory         |International TV Shows|
+---------------------+--------

In [13]:
from pyspark.sql import Window
from pyspark.sql.functions import col, rank, count

# Window Function untuk ranking director
window_dir = Window.partitionBy().orderBy(col("total_films").desc())
df_director = df_netflix.groupBy("director").count().withColumnRenamed("count","total_films")
df_director = df_director.withColumn("rank", rank().over(window_dir))
df_director.filter(col("rank") <= 5).show()

+--------------------+-----------+----+
|            director|total_films|rank|
+--------------------+-----------+----+
|                NULL|       2636|   1|
|       Rajiv Chilaka|         19|   2|
|Raúl Campos, Jan ...|         18|   3|
|        Marcus Raboy|         16|   4|
|         Suhas Kadav|         16|   4|
+--------------------+-----------+----+



In [23]:
# Running count jumlah film per tahun
window_year = Window.orderBy("release_year").rowsBetween(Window.unboundedPreceding, 0)
df_year_running = df_netflix.groupBy("release_year").count().orderBy("release_year") \
    .withColumn("running_total", count("count").over(window_year))
df_year_running.show(10)

+-----------------+-----+-------------+
|     release_year|count|running_total|
+-----------------+-----+-------------+
|             NULL|    2|            1|
|   Charles Rocket|    1|            2|
|          Dr. Dre|    1|            3|
|   Francis Weddey|    1|            4|
|     Imanol Arias|    1|            5|
|      Jade Eshete|    1|            6|
| Kristen Johnston|    1|            7|
| Marquell Manning|    1|            8|
|       Nick Kroll|    1|            9|
|    Nse Ikpe-Etim|    1|           10|
+-----------------+-----+-------------+
only showing top 10 rows



In [24]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Fungsi custom
def categorize_length(x):
    if x is None:
        return "unknown"
    elif x < 10:
        return "pendek"
    elif x <= 20:
        return "sedang"
    else:
        return "panjang"

udf_categorize = udf(categorize_length, StringType())
df_netflix = df_netflix.withColumn("length_category", udf_categorize("title_length"))
df_netflix.select("title", "title_length", "length_category").show(5, truncate=False)

+---------------------+------------+---------------+
|title                |title_length|length_category|
+---------------------+------------+---------------+
|Dick Johnson Is Dead |20          |sedang         |
|Blood & Water        |13          |sedang         |
|Ganglands            |9           |pendek         |
|Jailbirds New Orleans|21          |panjang        |
|Kota Factory         |12          |sedang         |
+---------------------+------------+---------------+
only showing top 5 rows



In [25]:
from pyspark.sql.functions import to_date, year

# Konversi date_added ke format tanggal
df_netflix = df_netflix.withColumn("date_added_clean", to_date("date_added", "MMMM d, yyyy"))
df_netflix = df_netflix.withColumn("year_added", year("date_added_clean"))

# Hitung jumlah film per tahun ditambahkan ke Netflix
df_netflix.groupBy("year_added").count().orderBy("year_added", ascending=False).show(10)

+----------+-----+
|year_added|count|
+----------+-----+
|      2021| 1491|
|      2020| 1872|
|      2019| 1997|
|      2018| 1623|
|      2017| 1162|
|      2016|  418|
|      2015|   72|
|      2014|   23|
|      2013|   10|
|      2012|    3|
+----------+-----+
only showing top 10 rows



In [26]:
# Genre terbanyak
df_genre.groupBy("genre").count().orderBy("count", ascending=False).show(10, truncate=False)

# Negara penyumbang film terbanyak
df_netflix.groupBy("country").count().orderBy("count", ascending=False).show(10, truncate=False)

# Tren tahun rilis
df_netflix.groupBy("release_year").count().orderBy("release_year", ascending=False).show(10)

+------------------------+-----+
|genre                   |count|
+------------------------+-----+
|International Movies    |2748 |
|Dramas                  |2419 |
|Comedies                |1670 |
|International TV Shows  |1350 |
|Documentaries           |866  |
|Action & Adventure      |857  |
|TV Dramas               |762  |
|Independent Movies      |751  |
|Children & Family Movies|641  |
|Romantic Movies         |616  |
+------------------------+-----+
only showing top 10 rows

+--------------+-----+
|country       |count|
+--------------+-----+
|United States |2805 |
|India         |972  |
|NULL          |832  |
|United Kingdom|419  |
|Japan         |245  |
|South Korea   |199  |
|Canada        |181  |
|Spain         |145  |
|France        |123  |
|Mexico        |110  |
+--------------+-----+
only showing top 10 rows

+-----------------+-----+
|     release_year|count|
+-----------------+-----+
|    United States|    1|
|    June 12, 2021|    1|
| January 15, 2021|    1|
| Januar